In [22]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet
import datetime
import ssl
import sys

In [6]:
target_shape = (200, 200)

In [31]:
x = tf.random.uniform((10,3))
tf.linalg.norm(x / tf.linalg.norm(x, axis=1, keepdims=1), axis=1)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.99999994, 1.        , 1.        , 1.        , 1.        ,
       1.        , 0.99999994, 1.        , 0.99999994, 0.99999994],
      dtype=float32)>

In [35]:
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        
        norm_anchor = anchor / tf.linalg.norm(anchor, axis=1, keepdims=1)
        norm_positive = positive / tf.linalg.norm(positive, axis=1, keepdims=1)
        norm_negative = negative / tf.linalg.norm(negative, axis=1, keepdims=1)
        
        ap_distance = tf.reduce_sum(tf.square(norm_anchor - norm_positive), -1)
        an_distance = tf.reduce_sum(tf.square(norm_anchor - norm_negative), -1)
        return (ap_distance, an_distance)


def get_model():
    # ResNet50
    base_cnn = resnet.ResNet50(
        weights="imagenet", input_shape=target_shape + (3,), include_top=False
    )

    # Add last layers
    flatten = layers.Flatten()(base_cnn.output)
    dense1 = layers.Dense(512, activation="relu")(flatten)
    dense1 = layers.BatchNormalization()(dense1)
    dense2 = layers.Dense(256, activation="relu")(dense1)
    dense2 = layers.BatchNormalization()(dense2)
    output = layers.Dense(256)(dense2)

    # Build model
    model = Model(base_cnn.input, output, name="model")

    # Freeze all weight till layer conv5_block1_out
    trainable = False
    for layer in base_cnn.layers:
        if layer.name == "conv5_block1_out":
            trainable = True
        layer.trainable = trainable

    # Inputs
    anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
    positive_input = layers.Input(name="positive", shape=target_shape + (3,))
    negative_input = layers.Input(name="negative", shape=target_shape + (3,))

    distances = DistanceLayer()(
        model(resnet.preprocess_input(anchor_input)),
        model(resnet.preprocess_input(positive_input)),
        model(resnet.preprocess_input(negative_input)),
    )

    # Siamese network
    siamese_network = Model(
        inputs=[anchor_input, positive_input, negative_input], outputs=distances
    )

    return siamese_network, model


In [36]:
siamese_network, embedding_model = get_model()

In [37]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        # an_distance >= ap_distance + 0.5
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

In [40]:
siamese_model = SiameseModel(siamese_network)
siamese_model.compile(optimizer=optimizers.Adam(0.0001))

In [41]:
siamese_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [21]:
siamese_model.load_weights('weights-epoch20')

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for weights-epoch20

In [ ]:
latest_checkpoint_path = "training_latest/weights.hdf5"
latest_checkpoint_dir = os.path.dirname(latest_checkpoint_path)
latest_cp_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_loss',
    filepath=latest_checkpoint_dir,
    save_weights_only=True,
    verbose=1)